[![Binder](https://mybinder.org/badge_logo.svg)](https://lab.mlpack.org/v2/gh/mlpack/examples/master?urlpath=lab%2Ftree%2Fforest_covertype_prediction_with_random_forests%2Fcovertype-rf-jl.ipynb)

In [ ]:
# @file covertype-rf-jl.ipynb
#
# Classification using Random Forest on the Covertype dataset.

In [11]:
using CSV
using DataFrames
using Libz
using mlpack

In [12]:
# Load the dataset from an online URL.
df = CSV.read(ZlibInflateInputStream(open(download(
    "https://lab.mlpack.org/data/covertype-small.csv.gz"))))

┌ Warning: `CSV.read(input; kw...)` is deprecated in favor of `DataFrame!(CSV.File(input; kw...))`
└ @ CSV /srv/julia/pkg/packages/CSV/W9RT2/src/CSV.jl:40
┌ Warning: `CSV.File` or `CSV.Rows` with `BufferedStreams.BufferedInputStream{Libz.Source{:inflate,BufferedStreams.BufferedInputStream{IOStream}}}` object is deprecated; pass a filename, `IOBuffer`, or byte buffer directly (via `read(x)`)
└ @ CSV /srv/julia/pkg/packages/CSV/W9RT2/src/utils.jl:239


,elevation,aspect,slope,horizontal_distance_to_hydrology,vertical_distance_to_hydrology
,Int64,Int64,Int64,Int64,Int64
1,3493,16,14,660,145
2,2917,4,15,525,135
3,3281,64,13,420,153
4,3021,90,14,0,0
5,3287,28,10,323,84
6,3093,25,14,90,17
7,3320,58,17,503,86
8,2968,140,12,108,16
9,2381,324,24,150,75


In [13]:
# Split the labels.
labels = df[!, :label][:]
dataset = select!(df, Not(:label))

,elevation,aspect,slope,horizontal_distance_to_hydrology,vertical_distance_to_hydrology
,Int64,Int64,Int64,Int64,Int64
1,3493,16,14,660,145
2,2917,4,15,525,135
3,3281,64,13,420,153
4,3021,90,14,0,0
5,3287,28,10,323,84
6,3093,25,14,90,17
7,3320,58,17,503,86
8,2968,140,12,108,16
9,2381,324,24,150,75


In [14]:
# Split the dataset using mlpack.
test, test_labels, train, train_labels = mlpack.preprocess_split(
    dataset,
    input_labels=labels,
    test_ratio=0.3)

([2933.0 203.0 … 0.0 0.0; 2926.0 278.0 … 0.0 0.0; … ; 2636.0 16.0 … 0.0 0.0; 2818.0 0.0 … 0.0 0.0], [2; 2; … ; 2; 2], [3322.0 294.0 … 1.0 0.0; 2867.0 154.0 … 0.0 0.0; … ; 3167.0 56.0 … 0.0 0.0; 2410.0 58.0 … 0.0 0.0], [1; 2; … ; 2; 3])

In [15]:
# Train a random forest.
rf_model, _, _ = mlpack.random_forest(training=train,
    labels=train_labels,
    print_training_accuracy=true,
    num_trees=10,
    minimum_leaf_size=3);

In [16]:
# Predict the labels of the test points.
_, predictions, _ = mlpack.random_forest(input_model=rf_model,
                                         test=test);

In [17]:
# Now print the accuracy.  The third return value ('probabilities'), which we
# ignored here, could also be used to generate an ROC curve.
correct = sum(predictions .== test_labels)
print("$(correct) out of $(length(test_labels)) test points correct " *
    "($(correct / length(test_labels) * 100.0)%).\n")

24708 out of 30000 test points correct (82.36%).
